In [1]:
from requests_html import HTMLSession
from bs4 import BeautifulSoup
import requests
import re
import numpy as np
import pandas as pd
import socket
from urllib3.connection import HTTPConnection

In [2]:
HTTPConnection.default_socket_options = (
    HTTPConnection.default_socket_options + [
        (socket.SOL_SOCKET, socket.SO_KEEPALIVE, 1),
        (socket.SOL_TCP, socket.TCP_KEEPIDLE, 45),
        (socket.SOL_TCP, socket.TCP_KEEPINTVL, 10),
        (socket.SOL_TCP, socket.TCP_KEEPCNT, 6)
    ] )

In [63]:
s = HTMLSession()

r = s.get('https://millercenter.org/the-presidency/presidential-speeches?field_president_target_id[8396]=8396')


soup= BeautifulSoup(r.text,'html.parser')


discursos = []
titulo= []
for speach in soup.find_all('div',class_='views-row') :
    link = speach.a.get('href')

    full_link = 'https://millercenter.org'+str(link)

    r_=s.get(full_link)
    
    soup_ =BeautifulSoup(r_.text,'html.parser')

    title = soup_.find('h2',class_='presidential-speeches--title').span.text
    
    titulo.append(title)

    full_discurso = soup_.find('div',class_= 'view-transcript')
    
    total = full_discurso.find_all('p')

    

    total = [elem.get_text() for elem in total]

    #taking out the list of list

    discursos.append(total)
    
    


In [64]:
res = [''.join(ele) for ele in discursos]

len(res)

12

In [39]:
len(discursos)

12

In [47]:
titulo

['January 19, 2021: Farewell Address',
 'January 13, 2021: Statement about the Violence at the Capitol',
 'January 7, 2021: Message After Pro-Trump Mob Overruns US Capitol',
 'January 6, 2021: Speech Urging Supporters to Go Home',
 'November 5, 2020: Remarks on the 2020 Election',
 'October 26, 2020: Swearing-In Ceremony of the Honorable Amy Coney Barrett to the US Supreme Court ',
 'September 26, 2020: Announcing His Nominee for the US Supreme Court',
 'September 7, 2020: Labor Day Press Conference',
 'August 8, 2020: Press Conference on Executive Orders',
 'July 4, 2020: Remarks at Salute to America',
 'June 20, 2020: Campaign Rally in Tulsa, Oklahoma',
 'June 13, 2020: Address at West Point Graduation']

In [49]:
fecha = [i.split(':')[0] for i in titulo]

discursos_titulo = [i.split(':')[1] for i in titulo]

In [50]:
fecha

['January 19, 2021',
 'January 13, 2021',
 'January 7, 2021',
 'January 6, 2021',
 'November 5, 2020',
 'October 26, 2020',
 'September 26, 2020',
 'September 7, 2020',
 'August 8, 2020',
 'July 4, 2020',
 'June 20, 2020',
 'June 13, 2020']

In [51]:
discursos_titulo

[' Farewell Address',
 ' Statement about the Violence at the Capitol',
 ' Message After Pro-Trump Mob Overruns US Capitol',
 ' Speech Urging Supporters to Go Home',
 ' Remarks on the 2020 Election',
 ' Swearing-In Ceremony of the Honorable Amy Coney Barrett to the US Supreme Court ',
 ' Announcing His Nominee for the US Supreme Court',
 ' Labor Day Press Conference',
 ' Press Conference on Executive Orders',
 ' Remarks at Salute to America',
 ' Campaign Rally in Tulsa, Oklahoma',
 ' Address at West Point Graduation']

In [65]:
df = pd.DataFrame({'titulo':discursos_titulo,'fecha':fecha,'discurso':res})

In [66]:
df

,titulo,fecha,discurso
0,Farewell Address,"January 19, 2021","My fellow Americans: Four years ago, we launch..."
1,Statement about the Violence at the Capitol,"January 13, 2021","My fellow Americans, I want to speak to you to..."
2,Message After Pro-Trump Mob Overruns US Capitol,"January 7, 2021",I would like to begin by addressing the heinou...
3,Speech Urging Supporters to Go Home,"January 6, 2021",I know your pain. I know your hurt.We had an e...
4,Remarks on the 2020 Election,"November 5, 2020",THE PRESIDENT: Good evening. I’d like to provi...
5,Swearing-In Ceremony of the Honorable Amy Con...,"October 26, 2020",THE PRESIDENT: Thank you very much. Appreci...
6,Announcing His Nominee for the US Supreme Court,"September 26, 2020",THE PRESIDENT: Thank you very much. Thank you....
7,Labor Day Press Conference,"September 7, 2020",THE PRESIDENT: Thank you very much. Thank you....
8,Press Conference on Executive Orders,"August 8, 2020","Thank you very much everybody. Thank you, and ..."
9,Remarks at Salute to America,"July 4, 2020",THE PRESIDENT: Wow. Are you having a good time...
